In [1]:
import pandas as pd
import dbconfig as dbConf

## 총합 양수 음수 갯수 구하는 함수

In [3]:
def pstv_ngtv_cnt(sum_num):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
            select sum_num
                 , sum(case when diff > 0 then 1 end) as pstv
                 , sum(case when diff < 0 then 1 end) as ngtv
              from (
            select a.*
                 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - sum_num from temp b where b.seq = (a.seq + 1)),0) as diff
              from (
                    select a.seq
                         , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
                         , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
                      from temp a
                   ) a
             where 1=1
               and sum_num = {sum_num}
                   ) b 
             group by sum_num
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴으로 결과값 뽑기

In [4]:
def Patten_GetNumber(max_val, f1, f2, f3, f4):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , sum(num)                                as num_sum
                  , ''                                      as bin
                  , sum(case when row_num = 1 then seq end) as f1
                  , sum(case when row_num = 2 then seq end) as f2
                  , sum(case when row_num = 3 then seq end) as f3
                  , sum(case when row_num = 4 then seq end) as f4
                  , sum(case when row_num = 5 then seq end) as f5
                  , sum(case when row_num = 6 then seq end) as f6
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
                  , seq
               from (
                    select seq, num   
                      from (
                            select 3 as seq, num
                              from in_list 
                             where seq = {max_val} 
                               and val = 3 
                             order by rand() 
                             limit {f1}
                           ) as t3
                    union all
                    select seq, num   
                      from (
                            select 5 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 5 
                             order by rand() 
                             limit {f2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (
                            select 10 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 10
                             order by rand() 
                             limit {f3}
                           ) as t10
                    union all
                    select seq, num   
                    from (
                          select 30 as seq, num
                            from in_list 
                           where seq = {max_val}
                             and val = 30 
                           order by rand() 
                           limit {f4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                 ) as b 
             ;
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## sql 다이렉트로 날릴수 있는 함수

In [5]:
def sql_direct(sql_list):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = sql_list

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴 얻기

In [6]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2 a
            order by 1 desc
            limit 10
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [14]:
# 패턴 뽑기
df = GetPatten()

stan    = list(range(0,1))
cnt3    = list(range(0,1))
cnt5    = list(range(0,1))
cnt10   = list(range(0,1))
cnt30   = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

In [15]:
df

,stan,3cnt,5cnt,10cnt,30cnt
0,1050,3,1,1,1
1,1049,2,1,2,1
2,1048,2,0,2,2
3,1047,2,2,1,1
4,1046,1,0,3,2
5,1045,1,1,2,2
6,1044,2,1,2,1
7,1043,3,1,0,2
8,1042,3,1,1,1
9,1041,0,2,2,2


## 전체 결과 

In [9]:
# 전체 내용
sql = """
select a.seq
     , a.n1
     , a.n2
     , a.n3
     , a.n4
     , a.n5
     , a.n6
     , a.sum_num as 합계
     , a.diff    as 차이    
     , b.pstv    as plus
     , b.ngtv    as minus
     , ''        as 빈칸
     , c.3cnt
     , c.5cnt
     , c.10cnt
     , c.30cnt
  from (
select a.seq
	 , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
	 , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
	 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) 
				 from temp b 
				where b.seq = (a.seq + 1)),0) as diff
  from temp a 
       ) a left outer join (select sum_num
								 , ifnull(sum(case when diff > 0 then 1 end),0) as pstv
								 , ifnull(sum(case when diff < 0 then 1 end),0) as ngtv
							  from (
							select (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
								 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) 
											 from temp b 
											where b.seq = (a.seq + 1)),0) as diff
							  from temp a
								   ) a
							 group by sum_num
						   ) b
					    on a.sum_num = b.sum_num 
		   left outer join in_list_2 c
                        on a.seq = c.stan 
 order by 1 desc
 limit 10
       
"""

sql_all_data = sql_direct(sql)

In [10]:
sql_all_data

,seq,n1,n2,n3,n4,n5,n6,합계,차이,plus,minus,빈칸,3cnt,5cnt,10cnt,30cnt
0,1050,6,12,31,35,38,43,165,0,8,9,,None,None,None,None
1,1049,3,5,13,20,21,37,99,66,7,1,,2,1,2,1
2,1048,6,12,17,21,32,39,127,-28,7,9,,2,0,2,2
3,1047,2,20,33,40,42,44,181,-54,0,3,,2,2,1,1
4,1046,7,16,25,29,35,36,148,33,2,11,,1,0,3,2
5,1045,6,14,15,19,21,41,116,32,11,5,,1,1,2,2
6,1044,12,17,20,26,28,36,139,-23,5,8,,2,1,2,1
7,1043,3,5,12,22,26,31,99,40,7,1,,3,1,0,2
8,1042,5,14,15,23,34,43,134,-35,8,7,,3,1,1,1
9,1041,6,7,9,11,17,18,68,66,2,0,,0,2,2,2


-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------

## 데이터 뽑기

In [18]:
# 최신 패턴으로 결과 뽑기
data = Patten_GetNumber(stan[1],0,0,0,0)

f1 = 3
f2 = 2
f3 = 1
f4 = 0

for i in range(0, 5):
    data = pd.concat([data, Patten_GetNumber(stan[1]+1, f1, f2, f3, f4)])
    #data = data.append(Patten_GetNumber(stan[1]+1, f1, f2, f3, f4), ignore_index = True)
    
data

,n1,n2,n3,n4,n5,n6,num_sum,bin,f1,f2,f3,f4,f5,f6
0,None,None,None,None,None,None,None,,None,None,None,None,None,None
0,12,17,32,33,34,40,168,,3,5,3,3,10,5
0,5,13,17,36,38,44,153,,3,3,5,10,3,5
0,2,5,32,36,38,44,157,,5,3,3,10,3,5
0,2,3,7,20,35,40,107,,5,3,10,3,3,5
0,2,12,17,31,41,43,146,,5,3,5,3,10,3


In [13]:
# 넘는 건만 담는다.
data2 = Patten_GetNumber(stan[1],0,0,0,0)

f1 = 3
f2 = 2
f3 = 1
f4 = 0

for i in range(0, 10):
    fill = Patten_GetNumber(stan[1]+1, f1, f2, f3, f4)
    
    if fill.loc[0]['num_sum'] > 128:
        #data2 = data2.append(fill)
        data2 = pd.concat([data2, fill])
    
data2

,n1,n2,n3,n4,n5,n6,num_sum,bin,f1,f2,f3,f4,f5,f6
0,None,None,None,None,None,None,None,,None,None,None,None,None,None
0,3,7,13,32,36,41,132,,3,5,3,3,5,10
0,11,16,32,33,35,37,164,,10,5,3,5,3,3
0,7,9,20,35,37,44,152,,5,10,3,3,3,5
0,17,23,36,37,38,42,193,,3,10,5,3,3,5
0,17,31,34,38,42,44,206,,3,3,10,3,5,5
